In [1]:
import requests
import json,base64,zlib,datetime
import math
import pandas as pd
import os

def decode_token(token):
    # base64解码
    token_decode = base64.b64decode(token.encode())
    # 二进制解压
    token_string = zlib.decompress(token_decode)
    return token_string

def encode_token(POI):
    ##结合武汉地区POI编号字符串
    strPOI = "https://wh.meituan.com/meishi/"+str(POI)+"/"
    # 生成token
    ts = int(datetime.datetime.now().timestamp() * 1000)
    token_dict = {
        'rId': 100900,
        'ver': '1.0.6',
        'ts': ts,
        'cts': ts + 100 * 1000,
        'brVD': [767,234],
        'brR': [[1536,864],[1536,834],24,24],
        'bI': [strPOI,""],
        'mT': ["304,119","304,116","306,114","308,111"],
        'kT': [],
        'aT': [],
        'tT': [],
        'aM': '',
        'sign': "eJwdjT1OAzEQhe+SwqV/lmSTjeQCpUJCdBxgsCfZEWt7ZY9BOUIuQkfBpXIOBqr36en9bKAiPEXvrB1VAEZhqwLx9QUS+vv31/3npiLljPVUeuZH5ioZVVam1NupRJSyKpUulF/r4mfmtR2N+Zx1QuIOWYeSjHCbybz9HRm1wkVqIpVl2LthVOsCfC41iV2pvT/jBy7CrVT2qjf8f+2dot/vJ+e2APEQwhbwPMHOarcbp4dhcsOgnbbabn4BsB1J/Q=="
    }
    # 二进制编码
    encode = str(token_dict).encode()
    # 二进制压缩
    compress = zlib.compress(encode)
    # base64编码
    b_encode = base64.b64encode(compress)
    # 转为字符串
    token = str(b_encode, encoding='utf-8')
    return token


token = encode_token("b1006")

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import math
import pandas as pd
import os
import uuid
 
def get_shop_page_response(page_num):
    simulateChromeBrowserData = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host':'sz.meituan.com',
        'Referer':'http://wh.meituan.com/meishi/',
        # 'Cookie': '_lxsdk_cuid=16212a00d8ec8-07cdb6596bad8e-178123e-1fa400-16212a00d8fc8; lat=22.780886; lng=113.906362; client-id=34908f62-ea11-4211-b60a-f62c32288b2e; uuid=9be4f96971ac4c9cab4c.1520730903.1.0.0; webloc_geo=22.527181%2C113.938582%2Cwgs84; ci=30; _lxsdk=16212a1cddfc8-011369480302e7-178123e-1fa400-16212a1cddfc8; __mta=247430459.1520730902128.1520731016684.1520731025187.5; _lxsdk_s=16212a00d8f-c83-6e-376%7C%7C9',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }

    purl='https://wh.meituan.com/meishi/api/poi/getPoiList'
    # token='eJxFj81uozAURl8lOxZEMQZi4kpdBIZRoGkJAdyWahY4QIAEwj9Jq3n3wUSdLiyde3w/6btfXG2E3MMMCgIWhPmM66N6HDm4EBaIG+e2Yb9LhGWEsShhKI3y8GMVQfi2tCa/Rv2hIGU+EyX5z+T2TH3ApYTmsxVi8nuYNkSZvWnVYJtc0rZl8wDAkCzyKG27oFgcLjkYuUlSQMeiCLBiHMvk7pSRBHkOIWb6juiOaET5jqsR4ZQ5TRlGwX9qf9wzO59lmvRYMI7MIcxcaK0/dTuJeGeoCDjnjptFtlZ6phaq12Ojxb7+2ir+O1Ytz+jsU6yWb4alVrWEogLTp3W609/6WoVkrftJNWyv2PYcvu8lCKSiTLfnbEe8qPM3y/TiExLk8KUr96Zzeyq97vwb1HIe5+1efOH9K7Q7w3p9j5xso9Eb/cSbfQ6HE6mMkpzFbZsQa2ykyTAlYtm7IFNvilYTV6yyeBXzYnhpQW/ykbje6faQ0GH3vLGC+kBLOUwOjXUsKA1oIauNCk1gPz5yf/8Bg1GdFg=='

    puuid=uuid.uuid1()
    params={'cityName':'武汉',
        'cateId':0,
            'areaId':1006,
            'sort':'',
            'cityName': '武汉',
            'dinnerCountAttrId': '',
            'page': int(page_num),
            'uuid': puuid,
            'userId': 0,
            'platform': 1,
            'partner': 126,
            'originUrl': 'https://wh.meituan.com/meishi/b1006/',
            'riskLevel': 1,
            'optimusCode': 10,
            '_token': token
           }
    page = requests.get(purl,params=params,headers=simulateChromeBrowserData)
    return page

In [3]:
# 解析POI店铺第一页的JSON返回结果,获得一共有多少页店铺页面\
def get_shop_pages(poi_Shops_Json):
    data = json.loads(poi_Shops_Json)
    if 'data' in data.keys():
        if 'totalCounts' in data.get('data'):
            shopCounts = data.get('data').get('totalCounts')
            # 美图一页是展示32个商铺
            pageNum = math.ceil(shopCounts/32)
            return pageNum
        else:
            return 1
    return 1
# 解析每一页店铺的页面信息以及店铺的POI
def parse_POI_Shop_Json(shop_json):
    one_page_data = pd.DataFrame()
    if 'data'in shop_json:
        for eachShopInfo in shop_json.get('data').get('poiInfos'):
            shopId = eachShopInfo.get('poiId')
            shopName = eachShopInfo.get('title')
            allCommentNum = eachShopInfo.get('allCommentNum')
            one_page_data = one_page_data.append(pd.DataFrame({'shopId':[shopId],'shopName':[shopName],'allCommentNum':[allCommentNum]}))
    return one_page_data

In [6]:
import pymysql
connect = pymysql.Connect(
        host='localhost', 
        port=3306, 
        user='root', 
        passwd='123456',
        db='douban', 
        charset='utf8')
cursor=connect.cursor()

SQL="""
DROP table IF EXISTS  Comments;"""
cursor.execute(SQL)
SQL="""
CREATE table Comments
(
 id int(10) unsigned NOT null ,
 shopname varchar(80) NOT NULL COMMENT '商家ID',
 allcomnum int(10) unsigned NOT NULL DEFAULT '0' COMMENT '最大评论数' ,
PRIMARY KEY (id) ,
UNIQUE KEY (id)
)ENGINE = InnoDB auto_increment=2 DEFAULT CHARSET=UTF8MB4 ; """
cursor.execute(SQL)


InternalError: (1050, "Table 'comments' already exists")

In [5]:
poiShopData = pd.DataFrame()
for i in range(10):
    page=get_shop_page_response(i+1)
    page_shop_json=json.loads(str(page.text))
    onePageData = parse_POI_Shop_Json(page_shop_json)
    poiShopData = poiShopData.append(onePageData)
print(poiShopData)
SQL="REPLACE  INTO Comments(id,shopname,allcomnum) VALUES(%s,%s,%s);"
for i in range(len(poiShopData)):
    shopId = int(poiShopData.iloc[i].shopId)
    shopName = poiShopData.iloc[i].shopName
    allCommentNum = int(poiShopData.iloc[i].allCommentNum)
    cursor.execute(SQL,(shopId,shopName,allCommentNum))
poiShopData.to_csv('poiShopsData.csv', index=False)

       shopId        shopName  allCommentNum
0   592422377         橘味·星选味道             17
0       74779    秀玉红茶坊（光谷二期店）          26304
0     2464754      捞旺猪肚鸡（光谷店）          19527
0   164168527         许多年以后烤吧           1026
0    42788407      鑫小城故事（光谷店）          11173
..        ...             ...            ...
0     5534854  南海底平价小海鲜（创业街店）           2238
0    51003002    木碳吧韩国烤肉（财大店）            868
0     4454197      momo时光欢乐餐厅           1420
0     5218756   江湖飞饼（澳门街美食城店）            113
0     5827508        天时饭庄罗田吊锅              4

[150 rows x 3 columns]


In [6]:
#关闭MYSQL连接
connect.close()
cursor.close()

In [10]:
    SQL="REPLACE  INTO Comments(id,shopname,allcomnum) VALUES(%s,%s,%s);"
    shopId = 111
    shopName = 'poiShopData.iloc[i].shopName'
    allCommentNum = 232424
    cursor.execute(SQL,(shopId,shopName,allCommentNum))

2

0